## 初期設定

In [1]:
from mymodule import MLs
from mymodule import myfunc as mf
import numpy as np
import pandas as pd
import glob
import os
from dotenv import load_dotenv
load_dotenv()
# root_dir = os.environ["root_dir"]
root_dir = "../"

データの参照先の設定、インスタンス作成

In [2]:
TARGET = "pressure" #ALTIMAGES等のディレクトリにくっつけます
#それぞれ1番最後の'/'は忘れずつけること
ALTIMAGES0 = root_dir + "imgout/0106ML0/"#リコネクションなし分割済データフォルダ.makesepnpy.ipynbであらかじめ分割データを入れておく
ALTIMAGES1 = root_dir + "imgout/0106ML1X/"#リコネクションあり分割済データフォルダ.makesepnpy.ipynbであらかじめ分割データを入れておく
ALTIMAGES0 = root_dir + "imgout/0110_not/"#リコネクションなし分割済データフォルダ
ALTIMAGES1 = root_dir + "imgout/0110_1X/"#リコネクションあり分割済データフォルダ
LABEL_SOURCE0 = root_dir + "txt/labels0/"#リコネクションありのラベリングcsvのあるフォルダ
LABEL_SOURCE1 = root_dir + "txt/labels1X/"#リコネクションなしのラベリングcsvのあるフォルダ
IMGSHAPE = [10,100]#出来れば画像サイズはすべて同じで合ってほしい。違うサイズが混じる場合は最も多いサイズを指定すること
DO_PCA = False #データ圧縮。IMGSHAPEと、データサイズが途方もなく多ければTrueにする。
randomstate = None #学習時のパラメータ。初期値はNone。結果を固定したければ1とかに設定する。

ML = MLs.ML(TARGET, ALTIMAGES0, ALTIMAGES1, LABEL_SOURCE0, LABEL_SOURCE1, IMGSHAPE, DO_PCA,randomstate=randomstate)

FileNotFoundError: [Errno 2] No such file or directory: '../imgout/0106ML0/pressure/pressure_77.10.7_40.610.npy'

### 例

In [4]:
#学習
#モジュールに書いてるコードで実行
# model = ML.linearSVC()   
# ML.kneighbors()
# ML.rbfSVC()
# ML.XGBoost()

#or
#自由に記述し、MLクラスを利用して実行
from sklearn.svm import LinearSVC
def mylinearSVC(self):    
        model = LinearSVC(C=0.3, random_state=self.randomstate) # インスタンスを生成
        model.fit(self.X_train, self.y_train) # モデルの学習
        # 学習データに対する精度
        print("Train :", model.score(self.X_train, self.y_train)) 
        # テストデータに対する精度
        print("Test :", model.score(self.X_test, self.y_test)) 
        print(model.predict(self.X_test))
        pred = model.predict(self.X_test)
        svmres = pd.DataFrame(np.array([self.path_test, self.y_test, pred]).T, columns=["path", "y", "predict"])
        # print(classification_report(self.y_test, pred))
        return model
svm = mylinearSVC(ML)

Train : 0.9721957419764855
Test : 0.9560561299852289
[0. 0. 0. ... 1. 1. 1.]


c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [7]:
#参照可能なもの
ML.TARGET = TARGET#magfieldx, pressure,,,
ML.PATH1
ML.PATH0
ML.ALLTARINDATA0#データ参照先フォルダ
ML.ALLTARINDATA1
ML.PATH0TEST#テストデータ一覧
ML.PATH1TEST
ML.PATH0TRAIN#学習データ
ML.PATH1TRAIN
ML.X_test
ML.y_test
ML.path_test
ML.X_train
ML.y_train
ML.path_train
#後から変更可能なもの
ML.randomstate = 1

['../imgout/test1227/magfieldx/magfieldx_49.10.3_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.6_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.12.13_147.452.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.13.13_42.607.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.7.13_127.625.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.4_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.13.13_102.420.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.3_82.620.npy']

In [7]:
#学習モデルをファイルに保存、読み込み例
import pickle
path = "pickletest"

file = open(path, "wb") 
pickle.dump(svm, file)
file.close

with open(path, 'rb') as f:
    dict = pickle.load(f)
print(svm)

LinearSVC(C=0.3)


## 以下自由に

In [5]:
model = ML.XGBoost()  

Train : 0.9988993710691824
Test : 0.9974433893352812
[1 0 0 ... 1 1 1]
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       735
         1.0       1.00      1.00      1.00      2003

    accuracy                           1.00      2738
   macro avg       1.00      1.00      1.00      2738
weighted avg       1.00      1.00      1.00      2738



In [6]:
import pickle
picout = root_dir + "MLres/"

for target in ["density", "enstrophy", "pressure","magfieldx", "magfieldy", "velocityx","velocityy"]:
    for tag, altimages1, labelsources1 in [("X","imgout/0110_1X/","txt/labels1X/"), ("O","imgout/0110_1O/", "txt/labels1O/")]:
        TARGET = target
        #それぞれ1番最後の'/'は忘れずつけること
        ALTIMAGES0 = root_dir + "imgout/0110_not/"#リコネクションなし分割済データフォルダ
        ALTIMAGES1 = root_dir + altimages1#リコネクションあり分割済データフォルダ
        LABEL_SOURCE0 = root_dir + "txt/labels0/"#リコネクションありのラベリングcsvのあるフォルダ
        LABEL_SOURCE1 = root_dir + labelsources1#リコネクションなしのラベリングcsvのあるフォルダ
        IMGSHAPE = [10,100]#出来れば画像サイズはすべて同じで合ってほしい。違うサイズが混じる場合は最も多いサイズを指定すること
        DO_PCA = False #データ圧縮。IMGSHAPEと、データサイズが途方もなく多ければTrueにする。
        randomstate = None #学習時のパラメータ。初期値はNone。結果を固定したければ1とかに設定する。

        ML = MLs.ML(TARGET, ALTIMAGES0, ALTIMAGES1, LABEL_SOURCE0, LABEL_SOURCE1, IMGSHAPE, DO_PCA,randomstate=randomstate)
        summary = open(picout+"summary.txt", "a")
        def savemodel(model, methodname, summary=summary):
            picout = root_dir + "MLres/"
            path = picout + "model_" + target + "_" + tag + "_" + methodname
            # file = open(path, "wb") 
            # pickle.dump(model, file)
            # file.close()
            print("model_" + target + "_" + tag + "_" + methodname+"\n")
            print("------------------\n\n\n\n")
        model = ML.linearSVC()   
        savemodel(model, "linearSVC")
        model = ML.kneighbors()
        savemodel(model, "kneighbors")
        model = ML.rbfSVC()
        savemodel(model, "rbfSVC")
        model = ML.XGBoost()
        savemodel(model, "XGBoost")
        summary.close()

PCA 非実施


c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9155660377358491
Test : 0.8948137326515705
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.84      0.75      0.79       735
         1.0       0.91      0.95      0.93      2003

    accuracy                           0.89      2738
   macro avg       0.88      0.85      0.86      2738
weighted avg       0.89      0.89      0.89      2738

model_density_X_linearSVC

------------------




Train : 0.9556603773584905
Test : 0.9525200876552228
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.94      0.88      0.91       735
         1.0       0.96      0.98      0.97      2003

    accuracy                           0.95      2738
   macro avg       0.95      0.93      0.94      2738
weighted avg       0.95      0.95      0.95      2738

model_density_X_kneighbors

------------------




Train : 0.9551886792452831
Test : 0.9426588750913075
[0. 0. 0. ... 1. 1. 1.]
              

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 1.0
Test : 1.0
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       735
         1.0       1.00      1.00      1.00      1664

    accuracy                           1.00      2399
   macro avg       1.00      1.00      1.00      2399
weighted avg       1.00      1.00      1.00      2399

model_density_O_linearSVC

------------------




Train : 0.9668518186704892
Test : 0.9670696123384743
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.95       735
         1.0       0.97      0.98      0.98      1664

    accuracy                           0.97      2399
   macro avg       0.97      0.96      0.96      2399
weighted avg       0.97      0.97      0.97      2399

model_density_O_kneighbors

------------------




Train : 0.9781401182583767
Test : 0.9766569403918299
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score 

...

Train : 0.9419458878337216
Test : 0.9053772405168821
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.86      0.83      0.84       735
         1.0       0.93      0.94      0.93      1664

    accuracy                           0.91      2399
   macro avg       0.89      0.88      0.89      2399
weighted avg       0.90      0.91      0.90      2399

model_velocityx_O_linearSVC

------------------




Train : 0.8573732306038344
Test : 0.8549395581492288
[0. 1. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.77      0.75      0.76       735
         1.0       0.89      0.90      0.90      1664

    accuracy                           0.85      2399
   macro avg       0.83      0.83      0.83      2399
weighted avg       0.85      0.85      0.85      2399

model_velocityx_O_kneighbors

------------------




Train : 0.9111270381652034
Test : 0.882034180908712
[0. 1. 0. ... 1. 1. 1.]
           

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.994811320754717
Test : 0.9886778670562454
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       735
         1.0       0.99      0.99      0.99      2003

    accuracy                           0.99      2738
   macro avg       0.98      0.99      0.99      2738
weighted avg       0.99      0.99      0.99      2738

model_velocityy_X_linearSVC

------------------




Train : 0.8965408805031446
Test : 0.8962746530314097
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.80      0.82      0.81       735
         1.0       0.93      0.93      0.93      2003

    accuracy                           0.90      2738
   macro avg       0.87      0.87      0.87      2738
weighted avg       0.90      0.90      0.90      2738

model_velocityy_X_kneighbors

------------------




Train : 0.9682389937106918
Test : 0.9539810080350621
[0. 0. 0. ... 1. 1. 1.]
           

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9867407274681956
Test : 0.9749895789912464
[1. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96       735
         1.0       0.97      0.99      0.98      1664

    accuracy                           0.97      2399
   macro avg       0.98      0.96      0.97      2399
weighted avg       0.98      0.97      0.97      2399

model_velocityy_O_linearSVC

------------------




Train : 0.9347787134922056
Test : 0.9270529387244685
[1. 1. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.97      0.79      0.87       735
         1.0       0.91      0.99      0.95      1664

    accuracy                           0.93      2399
   macro avg       0.94      0.89      0.91      2399
weighted avg       0.93      0.93      0.92      2399

model_velocityy_O_kneighbors

------------------




Train : 0.9182942125067193
Test : 0.9099624843684869
[1. 1. 0. ... 1. 1. 1.]
          

In [9]:
def loadmodel(modelsdir, target, tag, methodname):
    path = picout + "model_" + target + "_" + tag + "_" + methodname
    with open(path, 'rb') as f:
        return pickle.load(f)


In [ ]:
modelspath = glob.glob(picout)
    